In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os,sys,inspect
import gc
from tqdm import tqdm
import random

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from load import *
from evals import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Embedding, Flatten, Multiply
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def leakyrelu(x, factor=0.2):
    return tf.maximum(x, factor*x)

## Load

In [3]:
df = load_data('../data/ml-100k/u.data', threshold=3)
uuid = df['userId'].unique()
uiid = df['movieId'].unique()


In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.15, random_state=SEED, stratify=df['userId'].values)

In [5]:
tr_X = np.stack([train['userId'].values.astype(np.int32), train['movieId'].values.astype(np.int32)], 1)
test_X = np.stack([test['userId'].values.astype(np.int32), test['movieId'].values.astype(np.int32)], 1)

tr_X.shape, test_X.shape

((85000, 2), (15000, 2))

## Model

In [6]:
class InnerProduct(layers.Layer):
    def __init__(self, x_dims):
        super().__init__()
        self.x_dims = x_dims
        
    def call(self, inputs):
        n = len(self.x_dims)
        
        p = []
        q = []
        for i in range(n):
            for j in range(i+1, n):
                p.append(i)
                q.append(j)
                
        p = tf.gather(inputs, p, axis=1)
        q = tf.gather(inputs, q, axis=1)
        
        out = p*q
        out = tf.squeeze(out, 1)
#         out = tf.reduce_sum(out, axis=2)
        return out
    
    
class OuterProduct(layers.Layer):
    def __init__(self, x_dims, kernel_type='mat'):
        super().__init__()
        self.x_dims = x_dims
        self.kernel_type = kernel_type
        
    def build(self, input_shape):
        n, m, k = input_shape
        
        if self.kernel_type == 'mat':
            self.kernel = self.add_weight(shape=(k, (m*(m-1)//2), k), 
                                         initializer = tf.zeros_initializer())
        else:
            self.kernel = self.add_weight(shape=((m*(m-1)//2), k),
                                         initializer = tf.zeros_initializer())
        
    def call(self, inputs):
        n = len(self.x_dims)
        
        p = []
        q = []
        for i in range(n):
            for j in range(i+1, n):
                p.append(i)
                q.append(j)
                
        p = tf.gather(inputs, p, axis=1)
        q = tf.gather(inputs, q, axis=1)
        
        if self.kernel_type == 'mat':
            kp = tf.transpose(tf.reduce_sum(tf.expand_dims(p, 1) * self.kernel, -1), [0, 2, 1])
            out = tf.reduce_sum(kp * q, -1)
        else:
            out = tf.reduce_sum(p * q * tf.expand_dims(self.kernel, 0), -1)
            
        return out

In [7]:
class PNN(Model):
    def __init__(self, x_dims, latent_dim, dnn_layers, model_type='inner', l2_emb=1e-4):
        super().__init__()
        self.x_dims = x_dims
        self.latent_dim = latent_dim

        self.embedding = Embedding(sum(x_dims)+1, latent_dim, input_length=1, embeddings_regularizer=l2(l2_emb))

        self.linear = Dense(latent_dim)

        if model_type == 'inner':
            self.pnn = InnerProduct(x_dims)
        elif model_type == 'outer':
            self.pnn = OuterProduct(x_dims)
        else:
            raise ValueError('no available model type')
        
        self.dnn = [Dense(unit, activation='relu') for unit in dnn_layers]
        
        self.final = Dense(1)
        
        self.flatten = Flatten()
        
    def call(self, inputs):
        emb = self.embedding(inputs + tf.constant((0, *np.cumsum(self.x_dims)))[:-1])
        
        linear = self.flatten(self.linear(emb))
        quadratic = self.pnn(emb)

        concat = tf.concat([linear, quadratic], -1)
        
        out = concat
        for layer in self.dnn:
            out = layer(out)
        
        out = self.final(out)
        return out
    

## Train

In [8]:
ipnn = PNN((len(uuid), len(uiid)), 8, [64, 32])
opnn = PNN((len(uuid), len(uiid)), 8, [64, 32], 'outer')

In [9]:
ipnn.compile(loss=losses.BinaryCrossentropy(from_logits=True), 
            optimizer=optimizers.Adam())

ipnn.fit(tr_X, 
       train['rating'].values,
      epochs=10,
      shuffle=True,
      validation_split=0.1)

Epoch 1/10
2391/2391 [==============================] - 8s 4ms/step - loss: 0.5956 - val_loss: 0.5767
Epoch 2/10
2391/2391 [==============================] - 8s 3ms/step - loss: 0.5608 - val_loss: 0.5746
Epoch 3/10
2391/2391 [==============================] - 8s 3ms/step - loss: 0.5534 - val_loss: 0.5728
Epoch 4/10
2391/2391 [==============================] - 8s 3ms/step - loss: 0.5472 - val_loss: 0.5703
Epoch 5/10
2391/2391 [==============================] - 8s 4ms/step - loss: 0.5408 - val_loss: 0.5708
Epoch 6/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5322 - val_loss: 0.5752
Epoch 7/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5251 - val_loss: 0.5807
Epoch 8/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5186 - val_loss: 0.5778
Epoch 9/10
2391/2391 [==============================] - 8s 3ms/step - loss: 0.5134 - val_loss: 0.5824
Epoch 10/10
2391/2391 [==============================] - 8s 3ms/step - loss: 0.507

In [10]:
opnn.compile(loss=losses.BinaryCrossentropy(from_logits=True), 
            optimizer=optimizers.Adam())

opnn.fit(tr_X, 
       train['rating'].values,
      epochs=10,
      shuffle=True,
      validation_split=0.1)

Epoch 1/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5968 - val_loss: 0.5802
Epoch 2/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5648 - val_loss: 0.5759
Epoch 3/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5552 - val_loss: 0.5743
Epoch 4/10
2391/2391 [==============================] - 10s 4ms/step - loss: 0.5503 - val_loss: 0.5706
Epoch 5/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5465 - val_loss: 0.5715
Epoch 6/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5406 - val_loss: 0.5745
Epoch 7/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5342 - val_loss: 0.5772
Epoch 8/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5279 - val_loss: 0.5753
Epoch 9/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5223 - val_loss: 0.5786
Epoch 10/10
2391/2391 [==============================] - 9s 4ms/step - loss: 0.5

## Eval

In [11]:
pred_i = ipnn.predict(test_X)
pred_o = opnn.predict(test_X)

In [12]:
print(np.sum(np.where(pred_i>0., 1, 0).flatten() == test['rating'].values) / len(pred_i))
print(np.sum(np.where(pred_o>0., 1, 0).flatten() == test['rating'].values) / len(pred_o))

0.7194666666666667
0.7258


In [13]:
from sklearn.metrics import precision_score, recall_score,  roc_auc_score, precision_recall_fscore_support

# inner
print(roc_auc_score(test['rating'].values, pred_i))
print(precision_score(test['rating'].values, np.where(pred_i>0., 1, 0)))
print(recall_score(test['rating'].values, np.where(pred_i>0., 1, 0)))

0.7878999244272435
0.7308988764044944
0.7820389516710747


In [14]:
# outer
print(roc_auc_score(test['rating'].values, pred_o))
print(precision_score(test['rating'].values, np.where(pred_o>0., 1, 0)))
print(recall_score(test['rating'].values, np.where(pred_o>0., 1, 0)))

0.7907344065609905
0.7306637410861218
0.8006732387593172
